In [1]:
#importing packages 
from lakeshore import Model350
import pandas as pd
from IPython.display import clear_output
import numpy as np
from serial import SerialException

import signal
import codecs

import serial
import os
import sys
import time
from time import sleep
# import Cryocode
import serial.tools.list_ports
from datetime import datetime,timedelta

import plotly.express as px
import plotly.graph_objects as go
from cryocooler_api import *

In [2]:
def lakeshore_initialize():
    try: 
        myinstrument =Model350()
        print("connection successful")
        return(myinstrument)
    except: 
        print("Device not found")
        

def get_temp_data():
    import datetime
    data = {
        'tempA_log': [],
        'tempB_log': [],
        'tempC_log': [],
        
        'tempD_log': [],
        'time':[]
        }
        
    # data['tempA_log'].append(np.float32(myinstrument.query('KRDG? A')))
    time.sleep(0.1)
    data['tempA_log'].append(np.float32(myinstrument.query('KRDG? A')))
    time.sleep(0.1)
    data['tempB_log'].append(np.float32(myinstrument.query('KRDG? B')))
    time.sleep(0.1)
    data['tempC_log'].append(np.float32(myinstrument.query('KRDG? C')))
    time.sleep(0.1)
    data['tempD_log'].append(np.float32(myinstrument.query('KRDG? D')))
    data['time'].append(datetime.datetime.now())
    return data
def log_temp_data_step_duration(step,duration):
    import datetime
    start_time=datetime.datetime.now()
    log_duration = float(duration) #seconds
    log_step=float(step) #seconds 
    df = pd.DataFrame(get_temp_data())
    while datetime.datetime.now()-start_time<datetime.timedelta(seconds=log_duration):
        # print("its true")
        df2=pd.DataFrame(get_temp_data())
        df=df.append(df2,ignore_index = True)
        sleep(log_step)
    return df
def log_temp_data_step_duration(step,duration):
    import datetime
    start_time=datetime.datetime.now()
    log_duration = float(duration) #seconds
    log_step=float(step) #seconds 
    df = pd.DataFrame(get_temp_data())
    while datetime.datetime.now()-start_time<datetime.timedelta(seconds=log_duration):
        # print("its true")
        df2=pd.DataFrame(get_temp_data())
        df=df.append(df2,ignore_index = True)
        sleep(log_step)
    return df

def start_cryocooler_pwmode(cryo_port='COM7', cooldown_power=30): 
    import datetime
    pidmode=0
    soft_stop_state=0
    cooldown_power=30 #W
    """To Start the cryocooler"""
    cryo_port='COM7'
    ser0=start_connection(cryo_port)
    set_pid_mode(0)
    set_target_power_output(cooldown_power)
    set_soft_stop_state(0)
    stop_connection(ser0)
    start_time=datetime.datetime.now()
    print(f"Cryocooler started on {start_time}")
    return start_time
    

def stop_cryocooler(cryo_port='COM7'):
    import datetime
    """Stop the cryocooler"""
    ser0=start_connection(cryo_port)
    set_soft_stop_state(1)
    stop_connection(ser0)
    stoptime=datetime.datetime.now()
    print(f"Cryocooler stopped at {stoptime}")
    return
    
def warmup_detector(temperature=290,heaterrange=5): 
    """Warm up with heater to 290 K"""
    myinstrument=lakeshore_initialize()
    myinstrument.command(f'SETP 1,{temperature}')
    time.sleep(1)
    myinstrument.device_serial.close()
    myinstrument.device_serial.open()
    myinstrument.command(f'RANGE 1,{heaterrange}')
    myinstrument.device_serial.close()
    print(f"CF1 target temperature set to {temperature} K. Heater on CF2 started with range = {heaterrange}")

In [3]:
def power_cycled_cooldown_parameters():
    cooldown_power=30 #W
    coldtip_temp_steps= [229.007,196.67,164.34,157.0] #in K  # [229.007,196.67,164.34,132.0]
    cf1_temp_steps=[255.844,223.51,191.18,188.42] # in K
    det_temp_steps=[280.0,247.7,215.3,193] # in K
    cooldown_time=[30,20,20,20] #in K #[25,20,20,20]
    dwell_time=[20,30,30,50] # in mins
    params={'cooldown_power':cooldown_power,'coldtip_temp_steps':coldtip_temp_steps,'cf1_temp_steps':cf1_temp_steps,'det_temp_steps':det_temp_steps,'cooldown_time':cooldown_time,'dwell_time':dwell_time}
    return params    

In [4]:
params=power_cycled_cooldown_parameters()

In [7]:
cryo_port='COM7'
cooldown_power=params['cooldown_power'] #W
coldtip_temp_steps= params['coldtip_temp_steps']#in K  # [229.007,196.67,164.34,132.0]
cf1_temp_steps=params['cf1_temp_steps'] # in K
det_temp_steps=params['det_temp_steps'] # in K
cooldown_time=params['cooldown_time'] #in K #[25,20,20,20]
dwell_time=params['dwell_time'] # in mins
start_time=datetime.datetime.now()
for idx,step in enumerate(coldtip_temp_steps):
    if idx==0: 
        print(f"Started cryo in Power mode, Power = {cooldown_power}")
        start_time=datetime.datetime.now()
        ser0=start_connection(cryo_port)
        set_target_power_output(30)
        set_pid_mode(0)
        set_soft_stop_state(0)
        stop_connection(ser0) 
        print(f"Setting CF1 setpoint to {cf1_temp_steps[idx]}")
        myinstrument=lakeshore_initialize()
        myinstrument.command(f'SETP 1,{cf1_temp_steps[idx]}')
        time.sleep(1)
        print("Setting CF2 heater range to Range5")
        myinstrument.device_serial.close()
        myinstrument.device_serial.open()
        myinstrument.command(f'RANGE 1,5')
        myinstrument.device_serial.close() 
        print(f"Cooling down for {cooldown_time[idx]} minutes")
        time.sleep(cooldown_time[idx]*60)
        #stop instead of setpoint temperature. 
        print(f"Stopping Cryocooler for {dwell_time[idx]}minutes")
        ser0=start_connection(cryo_port)
        set_soft_stop_state(1)
        stop_connection(ser0) 
        time.sleep((dwell_time[idx])*60)
    elif idx==1:
        print(f"Started cryo in Power mode, Power = {cooldown_power}")
        ser0=start_connection(cryo_port)
        set_target_power_output(30)
        set_pid_mode(0)
        set_soft_stop_state(0)
        stop_connection(ser0) 
        print(f"Setting CF1 setpoint to {cf1_temp_steps[idx]}")
        myinstrument=lakeshore_initialize()
        myinstrument.command(f'SETP 1,{cf1_temp_steps[idx]}')
        time.sleep(1)
        print("Setting CF2 heater range to Range5")
        myinstrument.device_serial.close()
        myinstrument.device_serial.open()
        myinstrument.command(f'RANGE 1,5')
        myinstrument.device_serial.close()
        print(f"Cooling down for {cooldown_time[idx]} minutes")
        time.sleep(cooldown_time[idx]*60)
        #stop instead of setpoint temperature. 
        ser0=start_connection(cryo_port)
        set_soft_stop_state(1)
        stop_connection(ser0) 
        print(f"Stopping Cryocooler for {dwell_time[idx]}minutes")
        time.sleep((dwell_time[idx])*60)
    elif idx==2: 
        print(f"Started cryo in Power mode, Power = {cooldown_power}")
        ser0=start_connection(cryo_port)
        set_target_power_output(30)
        set_pid_mode(0)
        set_soft_stop_state(0)
        stop_connection(ser0) 
        print(f"Setting CF1 setpoint to {cf1_temp_steps[idx]}")
        myinstrument=lakeshore_initialize()
        myinstrument.command(f'SETP 1,{cf1_temp_steps[idx]}')
        time.sleep(1)
        myinstrument.device_serial.close()
        myinstrument.device_serial.open()
        print("Setting CF2 heater range to Range5")
        myinstrument.command(f'RANGE 1,5')
        myinstrument.device_serial.close()
        print(f"Cooling down for {cooldown_time[idx]} minutes")
        time.sleep(cooldown_time[idx]*60)
        print(f"Switching to Temp control mode with temperature setpoint = {coldtip_temp_steps[idx]} K")
        ser0=start_connection(cryo_port)
        set_pid_mode(2)
        set_target_temperature(coldtip_temp_steps[idx])
        stop_connection(ser0) 
        print(f"Cryocooler in Temp control mode for {dwell_time[idx]}minutes")
        time.sleep(dwell_time[idx]*60)
    elif idx==3: 
        print(f"Started cryo in Power mode, Power = {cooldown_power}")
        ser0=start_connection(cryo_port)
        set_target_power_output(30)
        set_pid_mode(0)
        set_soft_stop_state(0)
        stop_connection(ser0) 
        print(f"Setting CF1 setpoint to {cf1_temp_steps[idx]}")
        myinstrument=lakeshore_initialize()
        myinstrument.command(f'SETP 1,{cf1_temp_steps[idx]}')
        time.sleep(1)
        print("Setting CF2 heater range to Range5")
        myinstrument.device_serial.close()
        myinstrument.device_serial.open()
        myinstrument.command(f'RANGE 1,5')
        myinstrument.device_serial.close()
        print(f"Cooling down for {cooldown_time[idx]} minutes")
        time.sleep(cooldown_time[idx]*60)
        print(f"Switching to Temp control mode with temperature setpoint = {coldtip_temp_steps[idx]} K")
        ser0=start_connection(cryo_port)
        set_pid_mode(2)
        set_target_temperature(coldtip_temp_steps[idx])
        stop_connection(ser0) 
        time.sleep(dwell_time[idx]*60)
        print(f"Will have to be shut down with manual command")

Started cryo in Power mode, Power = 30
Connected


NameError: name 'ser0' is not defined

In [5]:
ser0=start_connection('COM7')

Connected


In [6]:
stop_connection(ser0)

Connection stopped
